# Получение адресов, включенных в Программу реновации

**О реновации.** Реновация в Москве – это крупномасштабный проект по обновлению жилищного фонда города, который был запущен в 2017 году. Он предполагает снос старых и аварийных пятиэтажек и строительство вместо них новых комфортабельных домов. Проект реализуется на основе добровольного согласия жильцов, которые получают новое жилье взамен старого без доплаты. Реновация также предусматривает благоустройство территории, строительство новых социальных объектов и развитие инфраструктуры. 

Можно предполагать, в рамках задачи, чем лучше развита инфраструктура, чем больше социальных объектов и зеленых зон, тем привлекательнее район для жильцов, и, следовательно, тем выше может быть цена на жилье.

**Задача.** Получить данные с сайта с [Фонда реновации](https://fr.mos.ru/uchastnikam-programmy/karta-renovatsii/?category[]=ALL).

In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

import time
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [ ]:
#создаю экземпляр браузера Chrome, инициализирую драйвер Chrome
browser = Chrome("/Users/amalia/Desktop/chromedriver")
driver = webdriver.Chrome()

url = "https://fr.mos.ru/uchastnikam-programmy/karta-renovatsii/?category[]=ALL"
browser.get(url)

/var/folders/7l/jh1g9wm112171g1bjt4zytch0000gn/T/ipykernel_52461/1710455556.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome("/Users/amalia/Desktop/chromedriver")


In [ ]:
renov_df=[]
for i in range(1, 966):
    soup = BeautifulSoup(browser.page_source, 'lxml')
    objects = soup.findAll('tr', class_='object-item')
    for obj in objects:
        try:
            renov_address = obj.find('span', class_='table-title').text
        except AttributeError:
            renov_address = obj.find('a', class_='table-title').text
        renov_area = obj.find('div', class_='table-subtitle').text
        renov_status = obj.find('td', class_='object-statuses').find('span').text
        renov_df.append([renov_address, renov_area, renov_status])
    
    #если это шестая итерация на каждом круге, то клик на кнопку "Следующее"
    if i % 6 == 0:
        browser.find_element(By.XPATH, '//*[@id="main-map-wrapper"]/div[5]/div[2]/div[2]/ul/li[8]/span/span').click()
    else:
        browser.find_element(By.XPATH, f'/html/body/main/section[2]/div[5]/div[2]/div[2]/ul/li[{(i % 6) + 2}]').click()
    time.sleep(2)
    

In [ ]:
renov_df[-1]

In [ ]:
renov_df = pd.DataFrame(renov_df, columns = ['renov_address','renov_area', 'renov_status'])

In [ ]:
renov_df.to_csv("renovation.csv")

In [ ]:
df = pd.read_csv('realty_data.csv')

In [ ]:
renov_df = pd.read_csv("renovation.csv")

In [ ]:
df[['address_name']]

In [ ]:
renov_df[['renov_address']]

In [ ]:
renov_df['renov_address'] = renov_df['renov_address'].str.replace('дом', '')

In [ ]:
renov_df['renov_address'] = renov_df['renov_address'].str.replace('.', '')

In [ ]:
renov_df = renov_df.rename(columns={"renov_address" : "address_name"})

In [ ]:
renov_df['address_name'] = renov_df['address_name'].str.replace('  ', ' ')

In [ ]:
renov_df = df.merge(renov_df, on = "address_name", how="left")

In [ ]:
renov_df[renov_df['renov_status'].notnull()]

In [ ]:
renov_df.to_csv("renovation_realty_data.csv")